### Example PIC Code for the N-body Problem in Open Boundaries

In [1]:
%matplotlib qt

import numpy as np
from numba import jit, prange
from math import floor

from scipy.fft import fftn, ifftn, fftfreq
from scipy.special import sici

import matplotlib.pyplot as plt

Our first PIC code will be an N-body solver in a non-periodic box. As a test we will integrate the Keplerian two-body problem. Though a PIC code really isn't suited to a high-accuracy solution of such a simple problem,
the two-body problem will provide a fairly rigorous test.

We first include the cloud-in-cell assignment and interpolation routines
developed in the notebook ParticleInCell. Making interpolateJIT parallel is clearly not a very good idea here;
there just isn't enough work to make the overhead in setting up a parallel loop profitable for two iterations!

In [2]:
@jit(nopython=True)
def assignJIT(pos, X, h, grid):
    """
    Assign X to grid using inverse of tri-linear (cloud-in-cell) interpolation
    pos[N,:], input: positions of N particles
    X[N], input: quantity to be assigned
    grid[M,M,M], input: grid on which to assign
    h, input: grid cell width
    """
    M = grid.shape[0]
    grid[:] = 0
    for m in range(pos.shape[0]):
        # cell index of lower-left corner of point's cube:
        ix, iy, iz = int(floor(pos[m,0]/h-0.5)), int(floor(pos[m,1]/h-0.5)), int(floor(pos[m,2]/h-0.5))
        # next index in each direction
        ixp, iyp, izp = (ix+1)%M, (iy+1)%M, (iz+1)%M 
        # fraction of cube in x,y,z in the next cell
        ux = (pos[m,0]/h - 0.5 - ix)
        uy = (pos[m,1]/h - 0.5 - iy)
        uz = (pos[m,2]/h - 0.5 - iz)
        ix, iy, iz = ix%M, iy%M, iz%M

        # deposit particle's mass on grid
        grid[ix , iy , iz ] += (1-ux) * (1-uy) * (1-uz) * X[m]
        grid[ix , iy , izp] += (1-ux) * (1-uy) *   uz   * X[m]
        grid[ix , iyp, iz ] += (1-ux) *   uy   * (1-uz) * X[m]
        grid[ix , iyp, izp] += (1-ux) *   uy   *   uz   * X[m]
        grid[ixp, iy , iz ] +=   ux   * (1-uy) * (1-uz) * X[m]
        grid[ixp, iy , izp] +=   ux   * (1-uy) *   uz   * X[m]
        grid[ixp, iyp, iz ] +=   ux   *   uy   * (1-uz) * X[m]
        grid[ixp, iyp, izp] +=   ux   *   uy   *   uz   * X[m]

    # divide by cell volume
    grid /= h**3

    
@jit(nopython=True, parallel=True)
def interpolateJIT(grid, h, pos, X):
    """
    Interpolate from grid at pos onto X using tri-linear (cloud-in-cell) interpolation
    grid[M,M,M], input: grid from which to interpolate
    h, input: grid cell width
    pos[N,:], input: positions of N particles
    X[N], output: interpolated quantity
    """
    M = grid.shape[0]
    h3 = h**3
    X[:] = 0
    for m in prange(pos.shape[0]):
        ix, iy, iz = int(floor(pos[m,0]/h-0.5)), int(floor(pos[m,1]/h-0.5)), int(floor(pos[m,2]/h-0.5))
        ixp, iyp, izp = (ix+1)%M, (iy+1)%M, (iz+1)%M 
        ux = (pos[m,0]/h - 0.5 - ix)
        uy = (pos[m,1]/h - 0.5 - iy)
        uz = (pos[m,2]/h - 0.5 - iz)
        ix, iy, iz = ix%M, iy%M, iz%M

        X[m] = (\
                    grid[ ix,  iy,  iz] * (1-ux) * (1-uy) * (1-uz) + \
                    grid[ ix,  iy, izp] * (1-ux) * (1-uy) *   uz   + \
                    grid[ ix, iyp,  iz] * (1-ux) *   uy   * (1-uz) + \
                    grid[ ix, iyp, izp] * (1-ux) *   uy   *   uz   + \
                    grid[ixp,  iy,  iz] *   ux   * (1-uy) * (1-uz) + \
                    grid[ixp,  iy, izp] *   ux   * (1-uy) *   uz   + \
                    grid[ixp, iyp,  iz] *   ux   *   uy   * (1-uz) + \
                    grid[ixp, iyp, izp] *   ux   *   uy   *   uz \
                  ) * h3


Next is the class to solve Poisson's equation with open boundary conditions.

Usage is:   
`
 solver = SolveOpenPoisson()
 phi = solver(rho,L)
`

`solver` will only compute Ghat the first time it is called; subsequent calls will use the cached array.

We give two versions. The first is the spectrally convergent one from EllipticFourierOpenBoundaries:

In [3]:
class SolveOpenPoisson:
    def __init__(self):
        self.Ghat = None
        self.N = 0
        self.L = 0
        
    def __call__(self, rho, L, result):
        N = rho.shape[0]
        if self.Ghat is None or N != self.N or L != self.L:
            self.N = N
            self.L = L
            self.Ghat = self.spectralGhat(N,L)
            
        fargs = { 'workers':12, 'overwrite_x':True }
        result[:] = ifftn( self.Ghat * fftn(np.pad(-rho,(0,N)),**fargs), **fargs )[:N,:N,:N].real
    
    def spectralGhat(self,N,L):
        """
        Unbounded Laplace operater Fourier Green's function in 3D on a grid of N with
        side length L
        """
        pi = np.pi
        h = L/N
        sigma = h/pi 

        print("SolveOpenPoisson: generating spectral Ghat")
        
        i = np.arange(0,2*N)
        i = np.where(i>N,2*N-i,i)
        na = np.newaxis
        #grid of pi*r/h 
        rho = pi*np.sqrt(i[:,na,na]**2+i[na,:,na]**2+i[na,na,:]**2)
        fac = 1/(2*pi**2 * sigma)
        G = fac * sici(rho)[0]/(rho + 1e-38)
        G[0,0,0] = fac
        # transform into Fourier space
        Ghat = fftn(G)/G.size * (2*L)**3
        return Ghat

The second uses the finite-difference $\hat{G}(\mathbf{K})$ from the same notebook:

In [4]:
class SolveOpenPoissonFD:
    def __init__(self):
        self.Ghat = None
        self.N = 0
        self.L = 0
        
    def __call__(self, rho, L, result):
        N = rho.shape[0]
        if self.Ghat is None or N != self.N or L != self.L:
            self.N = N
            self.L = L
            self.Ghat = self.fdGhat(N,L)
            
        fargs = { 'workers':12, 'overwrite_x':True }
        result[:] = ifftn( self.Ghat * fftn(np.pad(-rho,(0,N)),**fargs), **fargs )[:N,:N,:N].real
    
    def fdGhat(self,N,L):
        """
        Unbounded Laplace operater Fourier Green's function in 3D on a grid of N with
        side length L
        """
        pi = np.pi
        h = L/N

        print("SolveOpenPoisson: generating FD Ghat")

        
        k = fftfreq(2*N,1)*pi
        na = np.newaxis
        Ghat = 1/((np.sin(k[ :,na,na])/(0.5*h))**2 + \
                  (np.sin(k[na, :,na])/(0.5*h))**2 + \
                  (np.sin(k[na,na, :])/(0.5*h))**2 + 1.0e-38)
        Ghat[0,0,0] = 0

        return Ghat

Fourth-order derivative function. We'll be lazy and use the periodic version here even though we aren't in a periodic box! As long as our particles stay away from the boundaries, we'll be fine. Terrible idea for a production code, of course!

Again, parallelism here will slow things down, but would be beneficial at large $N$.

In [5]:
#def derivative(f, i, h):
#    """
#    Fourth-order finite difference approximation to first derivative in i-direction
#    """
#    return (-np.roll(f, 2, axis=i) + 8*np.roll(f, 1, axis=i) \
#           - 8*np.roll(f,-1, axis=i) + np.roll(f,-2, axis=i))/(12*h**4)

@jit(nopython=True, parallel=True)
def derivativeJIT(f, h, g):
    """
    Fourth-order finite difference approximation to first derivative on first index of f
    """
    M = f.shape[0]
    twh4 = 12*h**4
    for i in prange(M):
        ip2 = (i+2)%M
        ip1 = (i+1)%M
        im1 = (i-1)%M
        im2 = (i-2)%M
        g[i,:,:] = (-f[im2,:,:] + 8*f[im1,:,:] - 8*f[ip1,:,:] + f[ip2,:,:])/twh4
        
def derivative(f, d, h, g):
    """
    Fourth-order finite difference approximation to first derivative on d-th index of f
    """
    ff = f.swapaxes(0,d)
    gg = g.swapaxes(0,d)
    derivativeJIT(ff, h, gg)

The next ingredient is a container for the state of the system, the time, position, and velocities. It seemed logical to include routines here to perform the kick and drift operators, using another class (referenced here as `hamilton`) to solve for the acceleration.

In [6]:
class State:
    def __init__(self, time, position, velocity, mass):
        self.time = time
        self.position = position
        self.velocity = velocity
        self.mass = mass
        self.N = self.position.shape[0]

    def kick(self, h, hamilton, recalculate):
        # Update velocity 
        self.velocity += h * hamilton.momentumEquation(self, recalculate)
        return self
    
    def drift(self, h, hamilton):
        # Update positions
        self.position += h * hamilton.positionEquation(self)
        return self

Define an object which describes the grid; this will be convenient elsewhere.

In [7]:
class Grid:
    def __init__(self, dim, M, L):
        self.dim = dim
        self.M = M
        self.shape = (M,M,M)
        self.L = L           # length of a side
        self.h = L/M         # cell width

Now define a class which provides the functions used by `State` to implement kick and drift:

In [8]:
class PoissonPIC:
    def __init__(self, grid: Grid):
        self.g = grid
        self.rho = np.zeros(self.g.shape)
        self.phi = self.rho # same array as rho since we can overwrite rho with phi
        self.grad = np.zeros_like(self.rho)
        self.solver = SolveOpenPoisson()
        self.acc = None

        
    def positionEquation(self, state):
        # Return quantity to be multiplied by dt to update position: the velocity
        return state.velocity

    def momentumEquation(self, state, recalculate):
        # Return quantity to be multiplied by dt to update velocity: the acceleration
        
        # we need to calculate acc at the first step since we have no old value
        if recalculate or self.acc is None:
            if self.acc is None: self.acc = np.zeros_like(state.position)
            # assign particle mass distribution on rho grid
            assignJIT(state.position, state.mass, self.g.h, self.rho)
            # solve for potential
            self.solver(4*np.pi*self.rho, self.g.L, self.phi)
            # get components of acceleration
            for d in range(3):
                derivative(self.phi, d, self.g.h, self.grad)
                interpolateJIT(self.grad, self.g.h, state.position, self.acc[:,d])
            
        # return the acceleration       
        return self.acc 

Implement the kick-drift-kick Stormer-Verlet integration scheme using the machinery above:

In [9]:
def KDK(dt: float, hamilton: PoissonPIC, s: State) -> State:
    # only need to recalculate acc when the positions have changed
    s = s.kick(dt/2, hamilton, False).drift(dt, hamilton).kick(dt/2, hamilton, True)
    s.time += dt
    return s

***
***
Now that we have all of the ingredients, we need some initial conditions. The function sets up a two-body problem and returns an instance of `State` containging the initial conditions.

In [10]:
def twoBody(m1, m2, a_semi, ecc):
    """
    Get 2-body orbit
    """
    G = 1

    # first body is at rest at the origin
    
    # second body position and velocity at pericenter, so r is perpendicular to v
    mu = G*(m1+m2)
    r2 = a_semi*(1-ecc)
    v2 = np.sqrt(mu/a_semi * (1+ecc)/(1-ecc))

    mass = np.array([m1,m2])
    X = np.array([[0,0,0], [r2,0,0]])
    V = np.array([[0,0,0], [0,v2,0]])

    #Remove centre of mass position and velocity from particle data
    rsum = np.sum( mass[:,np.newaxis] * X, axis=0)
    vsum = np.sum( mass[:,np.newaxis] * V, axis=0)
    msum = np.sum(mass)
    rCOM = rsum/msum
    vCOM = vsum/msum
    X -= rCOM
    V -= vCOM

    # report the orbital period for convenience in setting the timestep
    period = np.sqrt(4*np.pi**2 * a_semi**3/mu)

    return State(time=0, position=X, velocity=V, mass=mass), period

Now we can put it all together!

In [11]:
# initial conditions
m1 = 1.0
m2 = 2.0
a_semi = 1.0
ecc = 0.2
state, period = twoBody(m1, m2, a_semi, ecc)

L = 2                 # length of box side
state.position += L/2 # put system in middle of box

# constant timestep is restictive for large eccentricities
# (variable timestep ruins symplecticity; this is only relevant for long duration runs)
dt = period/200

# set up the grid and get an instance of the Poisson solver
M = 64
grid = Grid(3, M, L)
hamilton = PoissonPIC(grid)

# prepare to animate the results
fig, ax = plt.subplots()
ax.set_aspect(1.0)
ax.set_xlim(0,grid.L)
ax.set_ylim(0,grid.L)

pts = []     # pts[j] is the plot instance for particle j's leading dot
tracks = []  # tracks[j] is the plot instance particle j's track
data = []    # data[j] are the coordinates of particle j's track
for i in range(state.N):
    tmp, = ax.plot(state.position[i,0], state.position[i,1])
    tracks.append(tmp)
    tmp, = ax.plot(state.position[i,0], state.position[i,1],'.')
    pts.append(tmp)
    data.append([[state.position[i,0]], [state.position[i,1]]])
    
steptxt = ax.text(0.2,1.1,f"step: {0:4d}   time: {0:8.2e}", transform=ax.transAxes)

steps = 0
while state.time <= 5*period:
    
    state = KDK(dt, hamilton, state) # take a KDK step
    steps += 1
    
    steptxt.set_text(f"step: {steps:4d}   time: {state.time/period:8.2e}")
    
    for j in range(state.N):
        data[j][0].append(state.position[j,0])
        data[j][1].append(state.position[j,1])
        tracks[j].set_data(data[j][0], data[j][1])
        pts[j].set_data(state.position[j,0],state.position[j,1])
    
    # don't steal window focus!
    #plt.pause(0.001)
    plt.gcf().canvas.draw_idle()
    plt.gcf().canvas.start_event_loop(0.001)

SolveOpenPoisson: generating spectral Ghat
